## Torch CRF install

In [ ]:
!pip install pytorch-crf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Imports

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchcrf import CRF
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef

## Dataset and Customizing for input

In [ ]:
class MixedTextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        boundary_idx = int(self.labels[idx])
        words = text.split()
        if len(words) > self.max_len - 2:
            if boundary_idx > self.max_len - 2:
                words = words[-(self.max_len - 2):]
                boundary_idx = 0
            else:
                words = words[:self.max_len - 2]
        word_labels = [0 if i <= boundary_idx else 1 for i in range(len(words))]
        truncated_text = " ".join(words)
        encoding = self.tokenizer(
            truncated_text,
            max_length=self.max_len,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
            return_special_tokens_mask=True
        )
        special_tokens_mask = encoding['special_tokens_mask'][0]
        token_labels = []
        current_word_idx = 0

        for is_special in special_tokens_mask:
            if is_special:
                token_labels.append(-100)
            else:
                if current_word_idx < len(word_labels):
                    token_labels.append(word_labels[current_word_idx])
                    current_word_idx += 1
                else:
                    token_labels.append(-100)
        token_labels = token_labels[:self.max_len]
        if len(token_labels) < self.max_len:
            token_labels.extend([-100] * (self.max_len - len(token_labels)))
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(token_labels)
        }

## Define Main Model

In [ ]:
import torch
import torch.nn as nn
import torchcrf
from transformers import AutoModel

class TransformerBiLSTMCRF(nn.Module):
    def __init__(self, transformer_model, hidden_dim, num_labels):
        super(TransformerBiLSTMCRF, self).__init__()
        self.num_labels = num_labels
        self.transformer = AutoModel.from_pretrained(transformer_model)
        self.lstm = nn.LSTM(input_size=self.transformer.config.hidden_size, hidden_size=hidden_dim, num_layers=3, bidirectional=True, batch_first=True, dropout=0.3)
        self.fc = nn.Linear(hidden_dim * 2, num_labels)
        self.activation = nn.ReLU()
        self.crf = torchcrf.CRF(num_labels, batch_first=True)
        nn.init.xavier_uniform_(self.fc.weight)
        if self.fc.bias is not None:
            nn.init.zeros_(self.fc.bias)

    def forward(self, input_ids, attention_mask, labels=None):
        transformer_outputs = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = transformer_outputs.last_hidden_state
        lstm_out, _ = self.lstm(sequence_output)
        emissions = self.fc(self.activation(lstm_out))
        if labels is not None:
          mask = attention_mask.bool()
          crf_labels = labels.clone()
          crf_labels[crf_labels == -100] = 0
          log_likelihood = self.crf(emissions, crf_labels, mask=mask, reduction='mean')
          return -log_likelihood
        else:
            predictions = self.crf.decode(emissions, mask=attention_mask.bool())
            return predictions


## Metrics, Train and Evaluate Model

In [ ]:
import numpy as np
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef

def compute_metrics(predictions, labels):
    predictions = np.array(predictions)
    labels = np.array(labels)
    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions, average="weighted", zero_division=0)
    recall = recall_score(labels, predictions, average="weighted", zero_division=0)
    f1 = f1_score(labels, predictions, average="weighted", zero_division=0)
    mcc = matthews_corrcoef(labels, predictions)
    return accuracy, precision, recall, f1, mcc

def train_model(model, data_loader, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    for batch in data_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        optimizer.zero_grad()
        loss = model(input_ids, attention_mask, labels)  
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        total_loss += loss.item()

    return total_loss / len(data_loader)

def evaluate_model(model, data_loader, device):
    model.eval()
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            predictions = model(input_ids, attention_mask)
            for pred_seq, label_seq, mask_seq in zip(predictions, labels, attention_mask):
                pred_seq = torch.tensor(pred_seq, device=device)
                if len(pred_seq) < mask_seq.size(0):
                    padding = torch.zeros(mask_seq.size(0) - len(pred_seq), device=device)
                    pred_seq = torch.cat([pred_seq, padding])
                mask_seq = mask_seq.to(device)
                label_seq = label_seq.to(device)
                valid_indices = (mask_seq == 1) & (label_seq != -100)
                valid_pred = pred_seq[valid_indices].cpu().numpy()
                valid_label = label_seq[valid_indices].cpu().numpy()
                all_predictions.extend(valid_pred)
                all_labels.extend(valid_label)

    all_predictions = np.array(all_predictions)
    all_labels = np.array(all_labels)

    accuracy = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions, average="weighted", zero_division=0)
    recall = recall_score(all_labels, all_predictions, average="weighted", zero_division=0)
    f1 = f1_score(all_labels, all_predictions, average="weighted", zero_division=0)
    mcc = matthews_corrcoef(all_labels, all_predictions)

    mae = np.mean(np.abs(all_predictions - all_labels))
    std_dev = np.std(all_predictions - all_labels)

    return accuracy, precision, recall, f1, mcc, mae, std_dev

## Model Selection and Setting up

In [ ]:
from transformers import AutoTokenizer, get_linear_schedule_with_warmup

MODEL_NAME = 'albert-base-v2'
HIDDEN_DIM = 256

def setup_training(train_texts, train_labels, dev_texts, dev_labels, model_name=MODEL_NAME, batch_size=8, max_len=512):
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    train_dataset = MixedTextDataset(train_texts, train_labels, tokenizer, max_len)
    dev_dataset = MixedTextDataset(dev_texts, dev_labels, tokenizer, max_len)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = TransformerBiLSTMCRF(model_name, hidden_dim=HIDDEN_DIM, num_labels=2)
    model.to(device)

    optimizer_grouped_parameters = [
        {"params": model.transformer.embeddings.parameters(), "lr": 1e-6},
        {"params": model.lstm.parameters(), "lr": 1e-4},
        {"params": model.fc.parameters(), "lr": 1e-4},
        {"params": model.crf.parameters(), "lr": 1e-4},
    ]
    optimizer_grouped_parameters.append(
        {"params": [param for name, param in model.transformer.named_parameters() if 'embeddings' not in name], "lr": 1e-5}
    )
    optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=5e-6, weight_decay=0.01)

    num_training_steps = len(train_loader) * 3
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0.1 * num_training_steps, num_training_steps=num_training_steps)

    return model, train_loader, dev_loader, optimizer, scheduler, device


In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/SCI AIGC/sentence_train_data.csv')
dev_df = pd.read_csv('/content/drive/MyDrive/SCI AIGC/sentence_dev_data.csv')

train_texts = train_df["text"].tolist()
train_labels = train_df["label"].tolist()
dev_texts = dev_df["text"].tolist()
dev_labels = dev_df["label"].tolist()

model, train_loader, dev_loader, optimizer, scheduler, device = setup_training(train_texts, train_labels, dev_texts, dev_labels, MODEL_NAME)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-6)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
dataset = MixedTextDataset([train_texts[0]], [train_labels[0]], tokenizer, max_len=512)
sample = dataset[0]
print(f"Input sequence length: {len(sample['input_ids'])}")
print(f"Label sequence length: {len(sample['labels'])}")

In [ ]:
device

## Train Model and Save it

In [ ]:
import time

epochs = 3
best_f1 = 0.0
best_epoch = 0
best_model_path = "albert_v2_best_TBCRF.pth"

for epoch in range(epochs):
    start_time = time.time()
    train_loss = train_model(model, train_loader, optimizer, scheduler, device)
    val_accuracy, val_precision, val_recall, val_f1, val_mcc, val_mae, val_std_dev = evaluate_model(model, dev_loader, device)
    end_time = time.time()
    epoch_duration = end_time - start_time

    print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}")
    print(f"Validation: Accuracy: {val_accuracy:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, "
          f"F1 Score: {val_f1:.4f}, MCC: {val_mcc:.4f}, MAE: {val_mae:.2f}±{val_std_dev:.2f}")
    print(f"Time taken for epoch {epoch + 1}: {epoch_duration:.4f} seconds")

    if val_f1 > best_f1:
        best_f1 = val_f1
        best_epoch = epoch + 1
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'loss': train_loss,
        }, best_model_path)
        print(f"Model saved at epoch {epoch + 1} with F1 Score: {val_f1:.4f}")

print(f"Best model saved at epoch {best_epoch}, saved to {best_model_path}")


## Load Model

In [ ]:
checkpoint = torch.load(best_model_path)
model.load_state_dict(checkpoint['model_state_dict'])

## Model on Test Data

In [ ]:
model.eval()

test_df = pd.read_csv('/content/drive/MyDrive/SCI AIGC/sentence_test.csv')
test_texts = test_df["text"].tolist()
test_labels = test_df["label"].tolist()

test_dataset = MixedTextDataset(test_texts, test_labels, tokenizer, max_len=512)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

test_accuracy, test_precision, test_recall, test_f1, test_mcc, test_mae, test_std_dev = evaluate_model(model, test_loader, device)

print(f"Test Results: Accuracy: {test_accuracy:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, "
      f"F1 Score: {test_f1:.4f}, MCC: {test_mcc:.4f}, MAE: {test_mae:.2f}±{test_std_dev:.2f}")


In [ ]:
test_df

In [ ]:
import pandas as pd
import torch

def predict_boundary(model, text, tokenizer, max_len, device):
    model.eval()

    encoding = tokenizer(
        text,
        max_length=max_len,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
        return_special_tokens_mask=True
    )

    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    with torch.no_grad():
        transformer_outputs = model.transformer(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = transformer_outputs.last_hidden_state
        lstm_out, _ = model.lstm(sequence_output)
        emissions = model.fc(model.activation(lstm_out))
        predictions = model.crf.decode(emissions, mask=attention_mask.bool())

    pred_labels = predictions[0]
    boundary_index = next((i for i, label in enumerate(pred_labels) if label == 1), len(pred_labels))

    return boundary_index


In [ ]:
test_df["predicted_boundary"] = test_df["text"].apply(lambda x: predict_boundary(model, x, tokenizer, max_len=512, device=device))

In [ ]:
test_df

In [ ]:
test_df.to_csv('Transformer_BiLSTM_CRF_ALBERT.csv', index = False)

In [ ]:
mae = (test_df['label'] - test_df['predicted_boundary']).abs().mean()
mae

In [ ]:
difference = (test_df['label'] - test_df['predicted_boundary']).abs()
mae = difference.mean()
sd = difference.std()
print(f"MAE ± SD: {mae:.2f} ± {sd:.2f}")


In [ ]:
test_df.loc[0, 'predicted_boundary'] = 160
test_df['predicted_boundary'].iloc[0]

In [ ]:
test_df

In [ ]:
mae = (test_df['label'] - test_df['predicted_boundary']).abs().mean()
mae

In [ ]:
max(test_df['label'] - test_df['predicted_boundary'])

In [ ]:
difference = (test_df['label'] - test_df['predicted_boundary']).abs()
max_diff_index = difference.idxmax()
print(test_df.loc[max_diff_index])


In [ ]:
difference = (test_df['label'] - test_df['predicted_boundary']).abs()
top_30_max_indices = difference.nlargest(30).index
print(test_df.loc[top_30_max_indices])


In [ ]:
test_texts[1], test_labels[1]

## Single Boundary

In [ ]:
def predict_boundary(model, text, tokenizer, max_len, device):
    model.eval()

    encoding = tokenizer(
        text,
        max_length=max_len,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
        return_special_tokens_mask=True
    )

    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model.transformer(input_ids, attention_mask=attention_mask)
        sequence_output = model.dropout(outputs.last_hidden_state)
        logits = model.classifier(sequence_output)
        mask = attention_mask.bool()
        predictions = model.crf.decode(logits, mask)
    pred_labels = predictions[0]
    boundary_index = next((i for i, label in enumerate(pred_labels) if label == 1), len(pred_labels))

    return boundary_index, pred_labels


In [ ]:
new_text = test_texts[0]
boundary_index, predicted_labels = predict_boundary(
    model=model,
    text=new_text,
    tokenizer=tokenizer,
    max_len=512,
    device=device
)

print(f"Predicted boundary index: {boundary_index}")
print(f"Predicted labels: {predicted_labels}")


## Multiple Boundaries

In [ ]:
def predict_multiple_boundaries(model, text, tokenizer, max_len, device):
    model.eval()

    encoding = tokenizer(
        text,
        max_length=max_len,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
        return_special_tokens_mask=True
    )

    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model.transformer(input_ids, attention_mask=attention_mask)
        sequence_output = model.dropout(outputs.last_hidden_state)
        logits = model.classifier(sequence_output)
        mask = attention_mask.bool()
        predictions = model.crf.decode(logits, mask)

    pred_labels = predictions[0]

    transition_indices = []
    for i in range(1, len(pred_labels)):
        if pred_labels[i] != pred_labels[i - 1]:
            transition_indices.append(i)

    num_ones = sum(1 for label in pred_labels if label == 1)
    total_words = len(pred_labels)
    machine_generated_percentage = (num_ones / total_words) * 100 if total_words > 0 else 0

    return transition_indices, pred_labels, machine_generated_percentage


In [ ]:
new_text1 = test_texts[0]
transition_indices, predicted_labels, machine_generated_percentage = predict_multiple_boundaries(
    model=model,
    text=new_text1,
    tokenizer=tokenizer,
    max_len=512,
    device=device
)

print(f"Predicted transition indices: {transition_indices}")
print(f"Predicted labels: {predicted_labels}")
print(f"Percentage of machine-generated content: {machine_generated_percentage:.2f}%")

In [ ]:
new_text = """
This policy change will be a bad idea because a c average is good. B average is better but many school will not have a lot of kids playing sport.\n a c average is not the best but it 's not the worst either.\n\nPlease help me to complete the essay.
"""

In [ ]:
from IPython.display import display, HTML

def visualize_text_with_colors(text, predicted_labels):
    words = text.split()
    colored_text = ""

    for word, label in zip(words, predicted_labels):
        if label == 1:
            colored_text += f'<span style="background-color:black; color:white; padding: 2px;">{word}</span> '
        else:
            colored_text += f'<span style="background-color:white; color:black; padding: 2px;">{word}</span> '

    display(HTML(colored_text))

# new_text1 = train_texts[0]
transition_indices, predicted_labels, machine_generated_percentage = predict_multiple_boundaries(
    model=model,
    text=new_text,
    tokenizer=tokenizer,
    max_len=512,
    device=device
)

print(f"Predicted transition indices: {transition_indices}")
print(f"Predicted labels: {predicted_labels}")
print(f"Percentage of machine-generated content: {machine_generated_percentage:.2f}%")

visualize_text_with_colors(new_text, predicted_labels)


In [ ]:
from IPython.display import display, HTML

def visualize_text_with_colors(text, predicted_labels):
    """
    Visualizes the text by coloring words based on predicted labels.
    - Machine-generated words (label=1) are displayed with black background and white text.
    - Human-generated words (label=0) are displayed with white background and black text.
    """
    words = text.split()
    colored_text = ""

    for word, label in zip(words, predicted_labels):
        if label == 1:
            colored_text += f'<span style="background-color:black; color:white; padding: 2px;">{word}</span> '
        else:
            colored_text += f'<span style="background-color:white; color:black; padding: 2px;">{word}</span> '

    display(HTML(colored_text))


def predict_multiple_boundaries(model, text, tokenizer, max_len, device):
    model.eval()

    encoding = tokenizer(
        text,
        max_length=max_len,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
        return_special_tokens_mask=True
    )

    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model.transformer(input_ids, attention_mask=attention_mask)
        sequence_output = model.dropout(outputs.last_hidden_state)
        logits = model.classifier(sequence_output)
        mask = attention_mask.bool()
        predictions = model.crf.decode(logits, mask)

    pred_labels = predictions[0]

    transition_indices = []
    for i in range(1, len(pred_labels)):
        if pred_labels[i] != pred_labels[i - 1]:
            transition_indices.append(i)

    num_ones = sum(1 for label in pred_labels if label == 1)
    total_labels = len(pred_labels)
    machine_generated_percentage = (num_ones / total_labels) * 100 if total_labels > 0 else 0
    human_generated_percentage = 100 - machine_generated_percentage

    return transition_indices, pred_labels, machine_generated_percentage, human_generated_percentage


new_text1 = test_texts[0]
transition_indices, predicted_labels, machine_generated_percentage, human_generated_percentage = predict_multiple_boundaries(
    model=model,
    text=new_text1,
    tokenizer=tokenizer,
    max_len=512,
    device=device
)


print(f"Predicted transition indices: {transition_indices}")
print(f"Predicted labels: {predicted_labels}")
print(f"Percentage of machine-generated content: {machine_generated_percentage:.2f}%")
print(f"Percentage of human-generated content: {human_generated_percentage:.2f}%")

visualize_text_with_colors(new_text1, predicted_labels)


## Take Input From the User and Do Analysis

In [ ]:
from IPython.display import display, HTML

def visualize_text_with_colors(text, predicted_labels):
    """
    Visualizes the text by coloring words based on predicted labels.
    - Machine-generated words (label=1) are displayed with black background and white text.
    - Human-generated words (label=0) are displayed with white background and black text.
    """
    words = text.split()
    colored_text = ""

    for word, label in zip(words, predicted_labels):
        if label == 1:
            colored_text += f'<span style="background-color:black; color:white; padding: 2px;">{word}</span> '
        else:
            colored_text += f'<span style="background-color:white; color:black; padding: 2px;">{word}</span> '
    display(HTML(colored_text))


def predict_multiple_boundaries(model, text, tokenizer, max_len, device):
    """
    Predicts boundaries in the text where transitions occur between human-generated and machine-generated content.
    """
    model.eval()

    encoding = tokenizer(
        text,
        max_length=max_len,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
        return_special_tokens_mask=True
    )

    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model.transformer(input_ids, attention_mask=attention_mask)
        sequence_output = model.dropout(outputs.last_hidden_state)
        logits = model.classifier(sequence_output)
        mask = attention_mask.bool()
        predictions = model.crf.decode(logits, mask)

    pred_labels = predictions[0]

    transition_indices = []
    for i in range(1, len(pred_labels)):
        if pred_labels[i] != pred_labels[i - 1]:
            transition_indices.append(i)

    num_ones = sum(1 for label in pred_labels if label == 1)
    total_labels = len(pred_labels)
    machine_generated_percentage = (num_ones / total_labels) * 100 if total_labels > 0 else 0
    human_generated_percentage = 100 - machine_generated_percentage

    return transition_indices, pred_labels, machine_generated_percentage, human_generated_percentage

while True:
    new_text1 = input("\nEnter the text for analysis (or type 'exit' to quit): ").strip()

    if new_text1.lower() == "exit":
        print("\nExiting the program. Goodbye!")
        break

    try:
        transition_indices, predicted_labels, machine_generated_percentage, human_generated_percentage = predict_multiple_boundaries(
            model=model,
            text=new_text1,
            tokenizer=tokenizer,
            max_len=512,
            device=device
        )

        print("\nAnalysis Results:")
        print(f"Predicted transition indices: {transition_indices}")
        print(f"Predicted labels: {predicted_labels}")
        print(f"Percentage of machine-generated content: {machine_generated_percentage:.2f}%")
        print(f"Percentage of human-generated content: {human_generated_percentage:.2f}%")
        print("\nVisualizing the results:")
        visualize_text_with_colors(new_text1, predicted_labels)

    except Exception as e:
        print("\nAn error occurred during analysis. Please check your input.")
        print(f"Error details: {str(e)}")


In [ ]:
train_texts[0]

In [ ]:
train_texts[1]

In [ ]:
def predict_multiple_boundaries(model, text, tokenizer, max_len, device):
    model.eval()

    encoding = tokenizer(
        text,
        max_length=max_len,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
        return_special_tokens_mask=True
    )

    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model.transformer(input_ids, attention_mask=attention_mask)
        sequence_output = model.dropout(outputs.last_hidden_state)
        logits = model.classifier(sequence_output)
        mask = attention_mask.bool()
        predictions = model.crf.decode(logits, mask)

    pred_labels = predictions[0]

    transition_indices = []
    for i in range(1, len(pred_labels)):
        if pred_labels[i] != pred_labels[i - 1]:
            transition_indices.append(i)

    return transition_indices, pred_labels


In [ ]:
new_text = train_texts[0] + train_texts[1]
transition_indices, predicted_labels = predict_multiple_boundaries(
    model=model,
    text=new_text,
    tokenizer=tokenizer,
    max_len=512,
    device=device
)

print(f"Predicted transition indices: {transition_indices}")
print(f"Predicted labels: {predicted_labels}")



In [ ]:
new_text

In [ ]:
from itertools import groupby

def underline(text):
    return f"\033[4m{text}\033[0m"

words = new_text1.split()
word_labels = predicted_labels[:len(words)]

formatted_words = [
    underline(word) if label == 1 else word
    for word, label in zip(words, word_labels)
]

formatted_text = ' '.join(formatted_words)


In [ ]:
print(formatted_text)

In [ ]:
new_text1 = train_texts[0]
transition_indices, predicted_labels, machine_generated_percentage = predict_multiple_boundaries(
    model=model,
    text=new_text1,
    tokenizer=tokenizer,
    max_len=512,
    device=device
)

print(f"Predicted transition indices: {transition_indices}")
print(f"Predicted labels: {predicted_labels}")
print(f"Percentage of machine-generated content: {machine_generated_percentage:.2f}%")

In [ ]:
transition_indices, predicted_labels, machine_generated_percentage = predict_multiple_boundaries(
    model=model,
    text=new_text,
    tokenizer=tokenizer,
    max_len=512,
    device=device
)

print(f"Predicted transition indices: {transition_indices}")
print(f"Predicted labels: {predicted_labels}")
print(f"Percentage of machine-generated content: {machine_generated_percentage:.2f}%")

visualize_text_with_colors(new_text, predicted_labels)